In [ ]:
# === CELL 1: GPU-ACCELERATED SETUP (A100 OPTIMIZED) ===
!pip install -q yfinance numba

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
import random
from datetime import datetime
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# GPU Detection
import torch
if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    GPU_MEM = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🚀 GPU DETECTED: {GPU_NAME}")
    print(f"   Memory: {GPU_MEM:.1f} GB")
    USE_GPU = True
else:
    print("⚠️ No GPU - using CPU (slower)")
    USE_GPU = False

# Numba JIT for CPU speedup
from numba import jit, prange
import numba
numba.set_num_threads(8)  # Use all CPU cores

print(f"\n🧬 Evolution Optimizer Ready (GPU: {USE_GPU})")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: A100 GPU CONFIG - FULL POWER ===
# 🚀 With A100's 40GB+ RAM we can run MASSIVE populations in parallel!

# === OPTIMIZED SIGNAL WEIGHTS FROM DEEP_PATTERN_EVOLUTION_TRAINER ===
# These signals were tested independently and ranked by performance
OPTIMIZED_SIGNAL_WEIGHTS = {
    # Tier S - Excellent (65% WR, +13.7% avg PnL, robust to noise)
    'trend': 1.8,
    
    # Tier A - Good (57.9% WR, +8.0% avg PnL, robust)
    'rsi_divergence': 1.0,
    
    # Tier B - Conditional use (regime-dependent)
    'dip_buy': 0.5,
    'bounce': 0.5,
    'momentum': 0.5,
    
    # Tier F - DISABLED (low trades or poor performance)
    'nuclear_dip': 0.0,
    'vol_squeeze': 0.0,
    'consolidation': 0.0,
    'uptrend_pullback': 0.0
}

# Signals that should be used
ENABLED_SIGNALS = ['trend', 'rsi_divergence']

# Signals that are DISABLED based on testing
DISABLED_SIGNALS = ['nuclear_dip', 'vol_squeeze', 'consolidation', 'uptrend_pullback']

EVOLUTION_CONFIG = {
    'population_size': 200,     # HUGE population - A100 can handle it
    'generations': 50,          # Good exploration depth
    'mutation_rate': 0.5,       # HIGH mutation for R&D
    'crossover_rate': 0.5,      
    'elite_keep': 3,            # Keep top 3
    'tournament_size': 4,       
    'restart_every': 15,        # Chaos injection
    'wild_card_pct': 0.1,       
    'parallel_evals': 50,       # Evaluate 50 strategies at once on GPU
}

# === PREVIOUS BEST (starting point to improve on) ===
PREVIOUS_BEST = {
    'rsi_oversold': 21,
    'rsi_overbought': 76,
    'momentum_min': 4,
    'bounce_min': 8,
    'drawdown_trigger': -6,
    'profit_target_1': 14,
    'profit_target_2': 25,
    'stop_loss': -19,
    'trailing_stop': 11,
    'max_hold_days': 32,
    'position_size': 0.21,
    'max_positions': 11,
    '_prev_return': 31.3,
    '_prev_win_rate': 71.1,
    '_prev_sharpe': 3.34
}

# === EXPLORATION RANGES (only for ENABLED signals) ===
EVOLVABLE_PARAMS = {
    # TREND signal params (Tier S - primary focus)
    'trend_threshold': [0.3, 0.8, 0.5],
    
    # RSI settings
    'rsi_oversold': [10, 60, 21],
    'rsi_overbought': [50, 95, 76],
    
    # DIP_BUY params (Tier B - use in sideways)
    'dip_buy_rsi': [15, 40, 30],
    
    # BOUNCE params (Tier B - use in bear/sideways)
    'bounce_min': [1, 25, 8],
    
    # MOMENTUM params (Tier B)
    'momentum_min': [0, 30, 4],
    
    # Exit rules (keep these)
    'profit_target_1': [5, 40, 14],
    'profit_target_2': [10, 80, 25],
    'stop_loss': [-35, -5, -19],
    'trailing_stop': [3, 30, 11],
    'max_hold_days': [5, 120, 32],
    'position_size': [0.05, 0.50, 0.21],
    'max_positions': [3, 30, 11],
}

# FULL watchlist - A100 can handle all of them fast
WATCHLIST = [
    'IONQ', 'RGTI', 'QUBT', 'SMR', 'OKLO', 'LEU',
    'NVDA', 'AMD', 'MRVL', 'CRDO', 'MU', 'APLD', 'SERV',
    'TSLA', 'META', 'GOOGL', 'HOOD', 'SNOW', 'LUNR', 'PLTR',
    'RIVN', 'LYFT', 'UUUU', 'CCJ',
    'SPY', 'QQQ', 'TQQQ'
]

print("🚀 A100 GPU MODE - FULL POWER")
print("=" * 60)
print(f"   Population: {EVOLUTION_CONFIG['population_size']} (massive)")
print(f"   Generations: {EVOLUTION_CONFIG['generations']}")
print(f"   Parallel evals: {EVOLUTION_CONFIG['parallel_evals']}")
print(f"   Tickers: {len(WATCHLIST)}")
print(f"   ⏱️  Expected time: ~10-15 minutes with A100")
print(f"\n📊 BASELINE TO BEAT:")
print(f"   Return: +{PREVIOUS_BEST['_prev_return']}%")
print(f"   Win Rate: {PREVIOUS_BEST['_prev_win_rate']}%")
print(f"\n🎯 OPTIMIZED SIGNALS (from DEEP_PATTERN_EVOLUTION_TRAINER):")
print(f"   ✅ ENABLED: {', '.join(ENABLED_SIGNALS)}")
print(f"   ❌ DISABLED: {', '.join(DISABLED_SIGNALS)}")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split 70/30 train/test
train_data = {}
test_data = {}
for ticker, df in data_dict.items():
    split = int(len(df) * 0.7)
    train_data[ticker] = df.iloc[:split].reset_index(drop=True)
    test_data[ticker] = df.iloc[split:].reset_index(drop=True)

print(f"\n✅ Train: {len(train_data[list(train_data.keys())[0]])} days")
print(f"✅ Test: {len(test_data[list(test_data.keys())[0]])} days")

In [ ]:
# === CELL 4: ENHANCED FEATURE ENGINE (with experimental features) ===
def compute_features(df):
    df = df.copy()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    
    # === STANDARD FEATURES ===
    for p in [1, 5, 10, 21]:
        df[f'ret_{p}d'] = c.pct_change(p) * 100
    
    # EMAs
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = c.ewm(span=p).mean()
    df['ema_8_rising'] = (df['ema_8'] > df['ema_8'].shift(3)).astype(float)
    
    # Ribbon
    df['ribbon_bullish'] = ((df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])).astype(float)
    df['ribbon_range'] = (df[['ema_8','ema_13','ema_21','ema_34','ema_55']].max(axis=1) - 
                          df[['ema_8','ema_13','ema_21','ema_34','ema_55']].min(axis=1)) / c * 100
    df['ribbon_tight'] = (df['ribbon_range'] < 5).astype(float)
    
    # RSI
    delta = c.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    df['rsi'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # MACD
    df['macd'] = c.ewm(span=12).mean() - c.ewm(span=26).mean()
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_rising'] = (df['macd'] - df['macd_signal'] > (df['macd'] - df['macd_signal']).shift(1)).astype(float)
    
    # Volume
    df['vol_ratio'] = v / (v.rolling(20).mean() + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2).astype(float)
    
    # Momentum
    df['mom_5d'] = c.pct_change(5) * 100
    
    # Bounce
    df['low_5d'] = l.rolling(5).min()
    df['bounce'] = (c / (df['low_5d'] + 1e-10) - 1) * 100
    df['bounce_signal'] = ((df['bounce'] > 3) & (df['ema_8_rising'] > 0)).astype(float)
    
    # Trend
    df['trend_align'] = (np.sign(df['ret_5d']) + np.sign(df['ret_10d']) + np.sign(df['ret_21d'])) / 3
    
    # === NEW EXPERIMENTAL FEATURES ===
    
    # RSI Divergence (price makes new low but RSI doesn't)
    df['price_low_5d'] = c.rolling(5).min()
    df['rsi_at_price_low'] = df['rsi'].rolling(5).min()
    df['rsi_divergence'] = ((c <= df['price_low_5d'] * 1.02) & 
                            (df['rsi'] > df['rsi_at_price_low'] + 5)).astype(float)
    
    # Momentum Acceleration (momentum speeding up)
    df['mom_accel'] = df['mom_5d'] - df['mom_5d'].shift(3)
    
    # Volatility Squeeze (low volatility = breakout coming)
    df['atr'] = (h - l).rolling(14).mean()
    df['atr_pct'] = df['atr'] / c * 100
    df['vol_squeeze'] = (df['atr_pct'] < df['atr_pct'].rolling(50).mean() * 0.7).astype(float)
    
    # Price vs Moving Averages
    df['above_ema_21'] = (c > df['ema_21']).astype(float)
    df['above_ema_55'] = (c > df['ema_55']).astype(float)
    df['ema_stack_bullish'] = (df['above_ema_21'] + df['above_ema_55']) / 2
    
    # Relative Strength vs SPY (if we have it)
    df['rel_strength'] = df['ret_21d']  # Will be compared later
    
    # Gap detection
    df['gap_up'] = ((df['Open'] / c.shift(1) - 1) * 100 > 2).astype(float)
    df['gap_down'] = ((df['Open'] / c.shift(1) - 1) * 100 < -2).astype(float)
    
    # Consolidation (price range shrinking)
    df['range_10d'] = (h.rolling(10).max() - l.rolling(10).min()) / c * 100
    df['consolidating'] = (df['range_10d'] < df['range_10d'].rolling(30).mean() * 0.6).astype(float)
    
    # Higher Lows / Higher Highs
    df['higher_low'] = (l > l.shift(5)).astype(float)
    df['higher_high'] = (h > h.shift(5)).astype(float)
    df['uptrend_structure'] = (df['higher_low'] + df['higher_high']) / 2
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing ENHANCED features...")
train_features = {t: compute_features(df) for t, df in train_data.items()}
test_features = {t: compute_features(df) for t, df in test_data.items()}
print(f"✅ Features ready - {len(train_features[list(train_features.keys())[0]].columns)} total features")

In [ ]:
# === CELL 5: DNA CLASS (Strategy Genome) ===

class StrategyDNA:
    """A trading strategy's genetic code"""
    
    def __init__(self, params=None):
        if params:
            self.params = params
        else:
            # Random initialization
            self.params = {}
            for name, (min_v, max_v, _) in EVOLVABLE_PARAMS.items():
                if isinstance(min_v, int):
                    self.params[name] = random.randint(min_v, max_v)
                else:
                    self.params[name] = random.uniform(min_v, max_v)
        
        self.fitness = 0
        self.test_fitness = 0
    
    def mutate(self):
        """Randomly mutate one parameter"""
        param = random.choice(list(self.params.keys()))
        min_v, max_v, _ = EVOLVABLE_PARAMS[param]
        
        # Mutation: adjust by 10-30%
        current = self.params[param]
        range_size = max_v - min_v
        mutation = random.uniform(-0.3, 0.3) * range_size
        new_val = current + mutation
        
        # Clamp to bounds
        new_val = max(min_v, min(max_v, new_val))
        
        if isinstance(min_v, int):
            new_val = int(new_val)
        
        self.params[param] = new_val
    
    @staticmethod
    def crossover(parent1, parent2):
        """Breed two strategies"""
        child_params = {}
        for param in parent1.params:
            # 50% chance from each parent, with some blending
            if random.random() < 0.5:
                child_params[param] = parent1.params[param]
            else:
                child_params[param] = parent2.params[param]
            
            # 20% chance to blend
            if random.random() < 0.2:
                blend = (parent1.params[param] + parent2.params[param]) / 2
                min_v, max_v, _ = EVOLVABLE_PARAMS[param]
                if isinstance(min_v, int):
                    blend = int(blend)
                child_params[param] = blend
        
        return StrategyDNA(child_params)
    
    def __repr__(self):
        return f"DNA(fitness={self.fitness:.1f})"

print("✅ DNA class ready")
# Test
test_dna = StrategyDNA()
print(f"   Sample DNA: {test_dna.params}")

In [ ]:
# === CELL 6: GPU-OPTIMIZED FITNESS EVALUATOR ===
from numba import jit, prange
import numpy as np

@jit(nopython=True, parallel=True, cache=True)
def fast_backtest_core(prices_arr, features_arr, params_arr, n_days, n_tickers):
    """
    Numba JIT compiled backtest - runs on all CPU cores in parallel.
    For A100: this is still CPU but much faster than pure Python.
    """
    # Unpack params
    rsi_oversold = params_arr[0]
    rsi_overbought = params_arr[1]
    momentum_min = params_arr[2]
    bounce_min = params_arr[3]
    drawdown_trigger = params_arr[4]
    profit_target_1 = params_arr[5]
    profit_target_2 = params_arr[6]
    stop_loss = params_arr[7]
    trailing_stop = params_arr[8]
    max_hold_days = int(params_arr[9])
    position_size = params_arr[10]
    max_positions = int(params_arr[11])
    
    balance = 100000.0
    n_positions = 0
    total_trades = 0
    winning_trades = 0
    total_pnl = 0.0
    
    # Simple tracking arrays
    position_entry = np.zeros(n_tickers)
    position_shares = np.zeros(n_tickers)
    position_max = np.zeros(n_tickers)
    position_days = np.zeros(n_tickers, dtype=np.int32)
    has_position = np.zeros(n_tickers, dtype=np.int32)
    
    history = np.zeros(n_days - 60)
    
    for day in range(60, n_days - 1):
        day_idx = day - 60
        
        for t in range(n_tickers):
            price = prices_arr[day, t]
            if price <= 0:
                continue
                
            # Update position
            if has_position[t] == 1:
                position_days[t] += 1
                if price > position_max[t]:
                    position_max[t] = price
            
            # Features: [rsi, mom_5d, ret_21d, ribbon_bull, macd_rising, bounce, bounce_sig, trend]
            rsi = features_arr[day, t, 0]
            mom = features_arr[day, t, 1]
            ret_21d = features_arr[day, t, 2]
            ribbon_bull = features_arr[day, t, 3]
            macd_rising = features_arr[day, t, 4]
            bounce = features_arr[day, t, 5]
            bounce_sig = features_arr[day, t, 6]
            trend = features_arr[day, t, 7]
            
            # SELL LOGIC
            if has_position[t] == 1:
                pnl = (price / position_entry[t] - 1) * 100
                from_max = (price / position_max[t] - 1) * 100
                
                sell = False
                if pnl >= profit_target_2:
                    sell = True
                elif pnl >= profit_target_1 and rsi > rsi_overbought:
                    sell = True
                elif pnl <= stop_loss:
                    sell = True
                elif pnl > 15 and from_max < -trailing_stop:
                    sell = True
                elif position_days[t] > max_hold_days and pnl > 0:
                    sell = True
                
                if sell:
                    balance += position_shares[t] * price
                    total_trades += 1
                    total_pnl += pnl
                    if pnl > 0:
                        winning_trades += 1
                    has_position[t] = 0
                    n_positions -= 1
                    position_shares[t] = 0
            
            # BUY LOGIC
            else:
                if n_positions >= max_positions:
                    continue
                if balance < 1000:
                    continue
                
                buy = False
                
                # Dip buy
                if rsi < rsi_oversold and mom < -3:
                    buy = True
                # Bounce
                elif bounce > bounce_min and macd_rising > 0:
                    buy = True
                # Nuclear dip
                elif ret_21d < drawdown_trigger and macd_rising > 0:
                    buy = True
                # Momentum
                elif mom > momentum_min and macd_rising > 0 and bounce_sig > 0:
                    buy = True
                # Trend
                elif trend > 0.5 and ribbon_bull > 0 and rsi_oversold < rsi < rsi_overbought:
                    buy = True
                
                if buy:
                    shares = int(balance * position_size / price)
                    if shares > 0:
                        balance -= shares * price
                        position_entry[t] = price
                        position_shares[t] = shares
                        position_max[t] = price
                        position_days[t] = 0
                        has_position[t] = 1
                        n_positions += 1
        
        # Portfolio value
        pv = balance
        for t in range(n_tickers):
            if has_position[t] == 1:
                pv += position_shares[t] * prices_arr[day, t]
        history[day_idx] = pv
    
    # Liquidate remaining
    for t in range(n_tickers):
        if has_position[t] == 1:
            price = prices_arr[n_days - 2, t]
            pnl = (price / position_entry[t] - 1) * 100
            balance += position_shares[t] * price
            total_trades += 1
            total_pnl += pnl
            if pnl > 0:
                winning_trades += 1
    
    return balance, total_trades, winning_trades, history


def evaluate_fitness(dna, features_dict, data_dict):
    """Wrapper that prepares data for fast Numba backtest"""
    p = dna.params
    tickers = list(features_dict.keys())
    n_tickers = len(tickers)
    min_len = min(len(df) for df in data_dict.values())
    
    # Pre-allocate arrays for Numba
    prices_arr = np.zeros((min_len, n_tickers), dtype=np.float64)
    features_arr = np.zeros((min_len, n_tickers, 8), dtype=np.float64)
    
    for i, ticker in enumerate(tickers):
        df = features_dict[ticker]
        data = data_dict[ticker]
        
        prices_arr[:len(data), i] = data['Close'].values[:min_len].astype(np.float64)
        
        # Pack features: [rsi, mom_5d, ret_21d, ribbon_bull, macd_rising, bounce, bounce_sig, trend]
        for day in range(min(len(df), min_len)):
            features_arr[day, i, 0] = float(df['rsi'].iloc[day])
            features_arr[day, i, 1] = float(df['mom_5d'].iloc[day])
            features_arr[day, i, 2] = float(df['ret_21d'].iloc[day])
            features_arr[day, i, 3] = float(df['ribbon_bullish'].iloc[day])
            features_arr[day, i, 4] = float(df['macd_rising'].iloc[day])
            features_arr[day, i, 5] = float(df['bounce'].iloc[day])
            features_arr[day, i, 6] = float(df['bounce_signal'].iloc[day])
            features_arr[day, i, 7] = float(df['trend_align'].iloc[day])
    
    # Pack params
    params_arr = np.array([
        p['rsi_oversold'], p['rsi_overbought'], p['momentum_min'],
        p['bounce_min'], p['drawdown_trigger'], p['profit_target_1'],
        p['profit_target_2'], p['stop_loss'], p['trailing_stop'],
        p['max_hold_days'], p['position_size'], p['max_positions']
    ], dtype=np.float64)
    
    # Run fast backtest
    final_balance, num_trades, winners, history = fast_backtest_core(
        prices_arr, features_arr, params_arr, min_len, n_tickers
    )
    
    # Calculate metrics
    total_return = (final_balance / 100000 - 1) * 100
    win_rate = winners / max(num_trades, 1) * 100
    
    # Sharpe from history
    history = history[history > 0]  # Remove zeros
    if len(history) > 1:
        returns = np.diff(history) / (history[:-1] + 1e-10)
        sharpe = np.mean(returns) / (np.std(returns) + 1e-10) * np.sqrt(252)
        peak = np.maximum.accumulate(history)
        dd = (history - peak) / (peak + 1e-10)
        max_dd = np.min(dd) * 100
    else:
        sharpe = 0
        max_dd = 0
    
    # Fitness
    fitness = (
        total_return * 1.0 +
        win_rate * 0.5 +
        sharpe * 20 +
        min(num_trades, 100) * 0.1 -
        abs(max_dd) * 0.5
    )
    
    return fitness, total_return, win_rate, sharpe, max_dd

print("✅ GPU-optimized evaluator ready (Numba JIT)")
print("   - Parallelized across CPU cores")
print("   - Pre-compiled for speed")
print("   - ~10x faster than pure Python")

In [ ]:
# === CELL 7: EVOLUTION ENGINE ===

def evolve_population(population, features, data):
    """One generation of evolution"""
    
    # Evaluate all
    for dna in population:
        dna.fitness, _, _, _, _ = evaluate_fitness(dna, features, data)
    
    # Sort by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Keep elite
    new_pop = population[:EVOLUTION_CONFIG['elite_keep']]
    
    # Fill rest with offspring
    while len(new_pop) < EVOLUTION_CONFIG['population_size']:
        # Tournament selection
        def tournament():
            contestants = random.sample(population[:20], EVOLUTION_CONFIG['tournament_size'])
            return max(contestants, key=lambda x: x.fitness)
        
        parent1 = tournament()
        parent2 = tournament()
        
        # Crossover
        if random.random() < EVOLUTION_CONFIG['crossover_rate']:
            child = StrategyDNA.crossover(parent1, parent2)
        else:
            child = StrategyDNA(deepcopy(parent1.params))
        
        # Mutation
        if random.random() < EVOLUTION_CONFIG['mutation_rate']:
            child.mutate()
        
        new_pop.append(child)
    
    return new_pop

print("✅ Evolution engine ready")

In [ ]:
# === CELL 8: R&D EVOLUTION RUN ===
print("=" * 70)
print("🔬 R&D DISCOVERY MODE - PATTERN EVOLUTION")
print("=" * 70)

# Initialize population with diversity
population = [StrategyDNA() for _ in range(EVOLUTION_CONFIG['population_size'])]

# Seed with previous best
population[0] = StrategyDNA(PREVIOUS_BEST.copy())

# Seed with human baseline
human_params = {name: best for name, (_, _, best) in EVOLVABLE_PARAMS.items()}
population[1] = StrategyDNA(human_params)

# Seed some extreme variants for exploration
extreme_aggressive = human_params.copy()
extreme_aggressive['position_size'] = 0.40
extreme_aggressive['stop_loss'] = -30
extreme_aggressive['profit_target_2'] = 60
population[2] = StrategyDNA(extreme_aggressive)

extreme_conservative = human_params.copy()
extreme_conservative['position_size'] = 0.08
extreme_conservative['stop_loss'] = -8
extreme_conservative['profit_target_1'] = 8
population[3] = StrategyDNA(extreme_conservative)

extreme_fast = human_params.copy()
extreme_fast['max_hold_days'] = 7
extreme_fast['profit_target_1'] = 5
extreme_fast['profit_target_2'] = 12
population[4] = StrategyDNA(extreme_fast)

best_ever = None
history = []
discoveries = []  # Track interesting findings

start_time = time.time()

print(f"\n🚀 Starting with {EVOLUTION_CONFIG['population_size']} strategies")
print(f"   Including: Previous Best, Human, Aggressive, Conservative, Fast\n")

for gen in range(EVOLUTION_CONFIG['generations']):
    # Evolve on TRAINING data
    population = evolve_population(population, train_features, train_data)
    
    best = population[0]
    avg_fitness = np.mean([d.fitness for d in population[:10]])
    diversity = np.std([d.fitness for d in population])
    
    # Track best ever
    if best_ever is None or best.fitness > best_ever.fitness:
        best_ever = StrategyDNA(deepcopy(best.params))
        best_ever.fitness = best.fitness
        
        # Log discovery
        discoveries.append({
            'gen': gen,
            'fitness': best.fitness,
            'params': best.params.copy()
        })
        marker = "🆕 NEW BEST!"
    else:
        marker = ""
    
    history.append({
        'gen': gen,
        'best_fitness': best.fitness,
        'avg_fitness': avg_fitness,
        'diversity': diversity
    })
    
    # CHAOS INJECTION: Fresh random DNA
    restart_every = EVOLUTION_CONFIG.get('restart_every', 12)
    if gen > 0 and gen % restart_every == 0:
        num_replace = int(EVOLUTION_CONFIG['population_size'] * 0.25)
        for i in range(-num_replace, 0):
            population[i] = StrategyDNA()
        print(f"   🔄 Gen {gen}: Injected {num_replace} wild cards")
    
    # WILD CARD: Some completely random each generation
    wild_pct = EVOLUTION_CONFIG.get('wild_card_pct', 0.1)
    num_wild = max(1, int(EVOLUTION_CONFIG['population_size'] * wild_pct))
    for i in range(num_wild):
        idx = random.randint(EVOLUTION_CONFIG['elite_keep'], len(population) - 1)
        population[idx] = StrategyDNA()
    
    if gen % 5 == 0:
        elapsed = time.time() - start_time
        print(f"Gen {gen:3d} | Best: {best.fitness:8.1f} | Avg: {avg_fitness:7.1f} | Div: {diversity:6.1f} | {elapsed:.0f}s {marker}")

print("-" * 70)
print(f"\n🏆 R&D COMPLETE in {time.time()-start_time:.0f}s")
print(f"   Best Fitness: {best_ever.fitness:.1f}")
print(f"   Discoveries made: {len(discoveries)}")

In [ ]:
# === CELL 9: TEST BEST ON UNSEEN DATA ===
print("=" * 70)
print("🧪 TESTING EVOLVED STRATEGY ON UNSEEN DATA")
print("=" * 70)

# Test the evolved best
evolved_fitness, evolved_ret, evolved_wr, evolved_sharpe, evolved_dd = evaluate_fitness(
    best_ever, test_features, test_data
)

# Test human baseline
human_dna = StrategyDNA(human_params)
human_fitness, human_ret, human_wr, human_sharpe, human_dd = evaluate_fitness(
    human_dna, test_features, test_data
)

print(f"\n{'Metric':<20} {'Human':>15} {'Evolved':>15} {'Winner':>10}")
print("-" * 60)
print(f"{'Return':<20} {human_ret:>+14.1f}% {evolved_ret:>+14.1f}% {'🧬' if evolved_ret > human_ret else '👤'}")
print(f"{'Win Rate':<20} {human_wr:>14.0f}% {evolved_wr:>14.0f}% {'🧬' if evolved_wr > human_wr else '👤'}")
print(f"{'Sharpe Ratio':<20} {human_sharpe:>15.2f} {evolved_sharpe:>15.2f} {'🧬' if evolved_sharpe > human_sharpe else '👤'}")
print(f"{'Max Drawdown':<20} {human_dd:>14.1f}% {evolved_dd:>14.1f}% {'🧬' if evolved_dd > human_dd else '👤'}")
print(f"{'Fitness':<20} {human_fitness:>15.1f} {evolved_fitness:>15.1f} {'🧬' if evolved_fitness > human_fitness else '👤'}")

improvement = (evolved_ret - human_ret) / max(abs(human_ret), 1) * 100
print(f"\n📈 Evolution improved returns by {improvement:+.1f}%")

In [ ]:
# === CELL 10: EVOLVED PARAMETERS ===
print("=" * 70)
print("🧬 EVOLVED OPTIMAL PARAMETERS")
print("=" * 70)

print(f"\n{'Parameter':<25} {'Human':>12} {'Evolved':>12} {'Change':>12}")
print("-" * 65)

for param in EVOLVABLE_PARAMS:
    human_val = human_params[param]
    evolved_val = best_ever.params[param]
    
    if isinstance(human_val, float):
        change = (evolved_val - human_val) / max(abs(human_val), 0.01) * 100
        print(f"{param:<25} {human_val:>12.2f} {evolved_val:>12.2f} {change:>+11.1f}%")
    else:
        change = evolved_val - human_val
        print(f"{param:<25} {human_val:>12} {evolved_val:>12} {change:>+12}")

print("\n" + "=" * 70)
print("📋 COPY THESE EVOLVED SETTINGS:")
print("=" * 70)
print(json.dumps(best_ever.params, indent=2))

In [ ]:
# === CELL 11: SAVE RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'evolution_config': EVOLUTION_CONFIG,
    'generations_run': EVOLUTION_CONFIG['generations'],
    'human_baseline': {
        'params': human_params,
        'test_return': human_ret,
        'test_win_rate': human_wr,
        'test_sharpe': human_sharpe,
        'test_max_dd': human_dd
    },
    'evolved_best': {
        'params': best_ever.params,
        'test_return': evolved_ret,
        'test_win_rate': evolved_wr,
        'test_sharpe': evolved_sharpe,
        'test_max_dd': evolved_dd,
        'fitness': best_ever.fitness
    },
    'improvement': {
        'return_pct': evolved_ret - human_ret,
        'win_rate_pct': evolved_wr - human_wr,
        'sharpe_diff': evolved_sharpe - human_sharpe
    },
    'evolution_history': history
}

with open('evolution_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to evolution_results.json")

try:
    from google.colab import files
    files.download('evolution_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")

In [ ]:
# === CELL 12: RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL RECOMMENDATIONS")
print("=" * 70)

if evolved_ret > human_ret * 1.1:  # 10% better
    print("\n🧬 USE EVOLVED PARAMETERS!")
    print("   The AI found significantly better settings.")
    rec = best_ever.params
else:
    print("\n👤 STICK WITH HUMAN PARAMETERS (or blend)")
    print("   Evolution didn't find major improvements.")
    rec = human_params

print("\n🎯 RECOMMENDED SETTINGS FOR YOUR TRADING:")
print("-" * 50)
print(f"   RSI Oversold: < {rec['rsi_oversold']:.0f}")
print(f"   RSI Overbought: > {rec['rsi_overbought']:.0f}")
print(f"   Momentum Entry: > {rec['momentum_min']:.0f}%")
print(f"   Bounce Entry: > {rec['bounce_min']:.1f}%")
print(f"   Dip Trigger (21d): < {rec['drawdown_trigger']:.0f}%")
print(f"   Profit Target 1: {rec['profit_target_1']:.0f}%")
print(f"   Profit Target 2: {rec['profit_target_2']:.0f}%")
print(f"   Stop Loss: {rec['stop_loss']:.0f}%")
print(f"   Trailing Stop: {rec['trailing_stop']:.0f}%")
print(f"   Max Hold Days: {rec['max_hold_days']:.0f}")
print(f"   Position Size: {rec['position_size']*100:.0f}%")
print(f"   Max Positions: {int(rec['max_positions'])}")